In [8]:
import dash
from dash import html, dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import numpy as np

In [9]:
# Import data
df = pd.read_csv('data/health-insurance-coverage-by-race-ethnicity-2022.csv')

In [10]:
# Verify data integrity
df.head()

,state,latitude,longitude,coverage_type,race_ethnicity,total
0,Alabama,32.806671,-86.79113,Insured,African-American / Black,1137663.0
1,Alabama,32.806671,-86.79113,Medicaid/CHIP,African-American / Black,303756.0
2,Alabama,32.806671,-86.79113,Medicare,African-American / Black,235067.0
3,Alabama,32.806671,-86.79113,Insured,American Indian / Alaska Native,10278.0
4,Alabama,32.806671,-86.79113,Medicaid/CHIP,American Indian / Alaska Native,1266.0


In [11]:
df.tail()

,state,latitude,longitude,coverage_type,race_ethnicity,total
1066,Wyoming,42.755966,-107.30249,Medicaid/CHIP,Other / Multiple Races,2874.0
1067,Wyoming,42.755966,-107.30249,Medicare,Other / Multiple Races,3418.0
1068,Wyoming,42.755966,-107.30249,Insured,White,420670.0
1069,Wyoming,42.755966,-107.30249,Medicaid/CHIP,White,28307.0
1070,Wyoming,42.755966,-107.30249,Medicare,White,103695.0


In [12]:
# Replace NaN values in 'number' with a default size (e.g., the median or a small number)
default_size = df['total'].median()
df['total'].fillna(default_size, inplace=True)

In [13]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [14]:
# App layout
app.layout = html.Div([
    html.H1("Health Insurance Coverage by Race and State"),
    dcc.Dropdown(
        id='race-dropdown',
        options=[{'label': race, 'value': race} for race in df['race_ethnicity'].unique()],
        value=df['race_ethnicity'].unique()[0],  # Default value
        clearable=False
    ),
    dcc.Graph(id='map-plot')
])

In [15]:
# Callback to update map based on selected race
@app.callback(
    Output('map-plot', 'figure'),
    [Input('race-dropdown', 'value')])
def update_map(selected_race):
    filtered_df = df[df['race_ethnicity'] == selected_race].copy()  # Create a copy to avoid modifying the original df
    if not filtered_df.empty:
        # Add random jitter to latitude and longitude to separate overlapping points
        jitter_amount = 0.6  # Adjust jitter amount based on visual preference and data density
        filtered_df['jittered_latitude'] = filtered_df['latitude'] + np.random.uniform(-jitter_amount, jitter_amount, size=len(filtered_df))
        filtered_df['jittered_longitude'] = filtered_df['longitude'] + np.random.uniform(-jitter_amount, jitter_amount, size=len(filtered_df))

        fig = px.scatter_mapbox(
            filtered_df,
            lat='jittered_latitude',
            lon='jittered_longitude',
            size='total',
            color='coverage_type',
            color_continuous_scale=px.colors.sequential.Viridis,
            zoom=3,
            height=600,
            custom_data=['state', 'coverage_type', 'total', 'race_ethnicity']
        )
        fig.update_traces(
            hovertemplate="<b>State:</b> %{customdata[0]}<br>" +
                          "<b>Coverage Type:</b> %{customdata[1]}<br>" +
                          "<b>Total:</b> %{customdata[2]:,}<br>" +
                          "<b>Race/Ethnicity:</b> %{customdata[3]}<extra></extra>"
        )
        fig.update_layout(
            mapbox_style="open-street-map",
            hoverlabel=dict(
                bgcolor="white",
                font_size=14,
                font_family="Arial"
            )
        )
    else:
        # Return an empty plot with a message if no data is available
        fig = px.scatter_mapbox()
        fig.update_layout(
            mapbox_style="open-street-map",
            margin={"r": 0, "t": 0, "l": 0, "b": 0},
            annotations=[
                {
                    "text": "No data available for selected race",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 16
                    }
                }
            ]
        )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)